In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
iris = load_iris()
X_train, X_test, Y_train, Y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=41)

# iris.data, iris.target

In [3]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        self.value = value

In [4]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):       
        X, Y = dataset[:,:-1], dataset[:,-1]
        n_samples, n_features = np.shape(X)
        
        if n_samples >= self.min_samples_split and curr_depth<=self.max_depth:
            best_split = self.get_best_split(dataset, n_samples, n_features)
            if best_split["info_gain"]>0:
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, n_samples, n_features):
        best_split = {
        'info_gain': 0,  
        'feature_index': None,
        'threshold': None,
        'dataset_left': None,
        'dataset_right': None
        }
        max_info_gain = -float("inf")
        for feature_index in range(n_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    curr_info_gain = self.information_gain(y, left_y, right_y)
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child):
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += - p_cls * np.log(p_cls)
        return entropy
        
    def calculate_leaf_value(self, Y):
        Y = list(Y)
        return max(Y, key=Y.count)    
    
    def fit(self, X, Y):
        Y = Y.reshape(-1, 1)
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)


In [21]:
tree = DecisionTreeClassifier(min_samples_split=3, max_depth=5)
tree.fit(X_train, Y_train)

Y_pred = tree.predict(X_test) 
print("accuracy:", accuracy_score(Y_test, Y_pred))

accuracy: 0.9333333333333333


In [18]:
import numpy as np
from scipy.stats import mode

class RandomForestClassifier:
    def __init__(self, n_trees=10, min_samples_split=3, max_depth=3, max_samples=None, max_features=None):
        self.n_trees = n_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.max_samples = max_samples if max_samples is not None else None
        self.max_features = max_features if max_features is not None else None
        self.trees = []
        self.feature_indices = []

    def fit(self, X, Y):
        n_rows, n_cols = X.shape
        n_samples = self.max_samples if self.max_samples is not None else n_rows
        n_features = self.max_features if self.max_features is not None else n_cols

        self.trees = []
        self.feature_indices = []
        for _ in range(self.n_trees):
            tree = DecisionTreeClassifier(min_samples_split=self.min_samples_split, max_depth=self.max_depth)
            indices = np.random.choice(n_rows, size=n_samples, replace=True)
            features_indices = np.random.choice(n_cols, size=n_features, replace=False)
            X_sample, Y_sample = X[indices][:, features_indices], Y[indices]
            tree.fit(X_sample, Y_sample)
            self.trees.append(tree)
            self.feature_indices.append(features_indices)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X[:, features]) for tree, features in zip(self.trees, self.feature_indices)])
        tree_preds = np.swapaxes(tree_preds, 0, 1)  
        majority_vote, _ = mode(tree_preds, axis=1)
        return majority_vote.flatten()

In [25]:
forest = RandomForestClassifier(n_trees=100, max_depth=3, max_samples=int(X_train.shape[0]/2), max_features=int(np.sqrt(X_train.shape[1])))
forest.fit(X_train, Y_train)

Y_pred = forest.predict(X_test)
print("accuracy:", accuracy_score(Y_test, Y_pred))

accuracy: 0.8333333333333334
